# Exploring Violent Crime Indicators in the Windy City

While comparing its level of violent crime to that in Iraq might be far-fetched, one thing that is for certain: the city of Chicago is plagued with deadly violence on a scale rarely seen in other cites across the US. 

The aim of this project is to answer the following questions: 

- Does temperature affect the rate of violent crime? (battery, criminal sexual assault, robbery, assualt, homicide)
- Which month of the year generally has the highest number of murders?
- What percentage of murders lead to an arrest? How often do criminals get away with murder?
- Where do violent crimes most frequently take place?
- Which districts have the highest and lowest arrest rates?

Understanding the answer to these questions can help police direct resources to where and when they are most likely to be needed, and can help citizens understand some of the patterns that may be occuring in their area. 

To answer our questions, we will be using data extracted from Chicago PD's CLEAR(Citizen Law Enforcement Analysis and Reporting) system. 
Our dataset reflects reported crimes from 2001-2017 and **contains 7,941,282 entries**. 

A few notes on potential issues with this data:

- Crime classifications could have been changed at a later date
- The data includes unverified reports that were given to Chicago PD
- The Chicago Police department doesn't guarantee the accuracy or completeness of the data

### Data Necessary for our Analysis

For our analysis we will simplfying our dataset and using only the following columns: 

- **`Case Number`**: Unique identifier for the record.
- **`Arrest`**: Indicates whether an arrest was made.
- **`Date`**: Date when the incident occured, or best estimate.
- **`Block`**: Partially redacted address where the incident occurred.
- **`Primary Type`**: The primary description of the IUCR code. For our purposes, the crime reported.
- **`Description`**: The secondary description-a subcategory of the primary type
- **`Location Description`**: A description of where the incident occurred, ie `STREET` or `RESIDENCE`.
- **`District`**: Indicates the [police district](https://data.cityofchicago.org/d/fthy-xz3r) where the incident occurred.
- **`Location`**: The location where the incident occurred in longitude and latitude, shifted for partial redaction, but falls on the same block.



### Reading in Our Dataset

Our data is contained in four separate .csv files reflecting years 2001-2004, 2005-2007, 2008-2011, and 2012-2016. We will be combining these files into one dataframe, `chicago`. 

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import glob 

# grab all .csv file names in our directory
files = glob.glob('Chicago*.csv')

# read in all 5 of the files and concatenate them together
chicago_data = [pd.read_csv(f, sep=',',on_bad_lines='skip',dtype=object) for f in files]
chicago = pd.concat(chicago_data)

In [7]:
chicago.shape

(7941282, 23)

We have nearly 8 million lines of data! We will be reducing columns to our core eight columns mentioned earlier.

In [8]:
chicago.head(3)

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,388,4785,HP610824,10/07/2008 12:39:00 PM,000XX E 75TH ST,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,...,6.0,69.0,01A,1178207.0,1855308.0,2008,08/17/2015 03:03:40 PM,41.758275857,-87.622451031,"(41.758275857, -87.622451031)"
1,835,4786,HP616595,10/09/2008 03:30:00 AM,048XX W POLK ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,...,24.0,25.0,01A,1144200.0,1895857.0,2008,08/17/2015 03:03:40 PM,41.87025207,-87.746069362,"(41.87025207, -87.746069362)"
2,1334,4787,HP616904,10/09/2008 08:35:00 AM,030XX W MANN DR,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,False,...,18.0,66.0,01A,1157314.0,1859778.0,2008,08/17/2015 03:03:40 PM,41.770990476,-87.698901469,"(41.770990476, -87.698901469)"


### Initial Impressions



In [4]:
chicago.describe()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Year,Updated On,Latitude,Longitude,Location,STATION,NAME,DATE,TAVG,TSUN
count,7941282,7941282,7941275,7941282,7941282,7941282,7941282,7941282,7939292,7941282,...,7941282,7941282,7835709,7835708,7835708,4507,4507,4507,1707,258
unique,6170812,6170812,6170472,2451622,58776,398,35,376,172,2,...,18,1310,838978,838432,840085,1,1,4507,93,1
top,443944,5430528,G592219,01/01/2008 12:01:00 AM,100XX W OHARE ST,0820,THEFT,SIMPLE,STREET,False,...,2008,02/04/2016 06:33:39 AM,41.976290414,-87.905227221,"(41.976290414, -87.905227221)",USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2004-01-01,72,0
freq,2,2,6,565,19927,611180,1640506,924724,2101842,5691862,...,852053,3574157,17378,17378,17378,4507,4507,1,46,258


Let's start with understanding our column formats.

In [9]:
chicago.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7941282 entries, 0 to 1456713
Data columns (total 23 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Unnamed: 0            object
 1   ID                    object
 2   Case Number           object
 3   Date                  object
 4   Block                 object
 5   IUCR                  object
 6   Primary Type          object
 7   Description           object
 8   Location Description  object
 9   Arrest                object
 10  Domestic              object
 11  Beat                  object
 12  District              object
 13  Ward                  object
 14  Community Area        object
 15  FBI Code              object
 16  X Coordinate          object
 17  Y Coordinate          object
 18  Year                  object
 19  Updated On            object
 20  Latitude              object
 21  Longitude             object
 22  Location              object
dtypes: object(23)
memory usage: 1.4+

We can see that all of our columns are in str format. Columns like `date` will need to be converted to `datetime` format, `Arrest` converted to `bool`. We will need to clean the `block` column by extracting the street name as well. We will rename our columns for clarity, and ensure capitalization in consistent. 

Let's take an initial look at some of our columns that will be core to our analysis.

In [10]:
chicago['Primary Type'].value_counts(dropna=False)

THEFT                                1640506
BATTERY                              1442716
CRIMINAL DAMAGE                       923000
NARCOTICS                             885431
OTHER OFFENSE                         491922
ASSAULT                               481661
BURGLARY                              470958
MOTOR VEHICLE THEFT                   370548
ROBBERY                               300453
DECEPTIVE PRACTICE                    280931
CRIMINAL TRESPASS                     229366
PROSTITUTION                           86401
WEAPONS VIOLATION                      77429
PUBLIC PEACE VIOLATION                 58548
OFFENSE INVOLVING CHILDREN             51441
CRIM SEXUAL ASSAULT                    29868
SEX OFFENSE                            28707
GAMBLING                               18806
LIQUOR LAW VIOLATION                   17513
INTERFERENCE WITH PUBLIC OFFICER       15710
ARSON                                  13097
HOMICIDE                                9051
KIDNAPPING

In [11]:
chicago['Arrest'].value_counts(normalize=True)

False    0.716743
True     0.283257
Name: Arrest, dtype: float64

Interesting to note that **between 2001 and 2016 only 28% of crimes reported resulted in an arrest!**

In [13]:
chicago.isnull().sum() / chicago.shape[0]

Unnamed: 0              0.000000e+00
ID                      0.000000e+00
Case Number             8.814698e-07
Date                    0.000000e+00
Block                   0.000000e+00
IUCR                    0.000000e+00
Primary Type            0.000000e+00
Description             0.000000e+00
Location Description    2.505893e-04
Arrest                  0.000000e+00
Domestic                0.000000e+00
Beat                    0.000000e+00
District                1.145911e-05
Ward                    8.817518e-02
Community Area          8.841028e-02
FBI Code                0.000000e+00
X Coordinate            1.329420e-02
Y Coordinate            1.329420e-02
Year                    0.000000e+00
Updated On              0.000000e+00
Latitude                1.329420e-02
Longitude               1.329433e-02
Location                1.329433e-02
dtype: float64

In [14]:
chicago.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7941282 entries, 0 to 1456713
Data columns (total 23 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Unnamed: 0            object
 1   ID                    object
 2   Case Number           object
 3   Date                  object
 4   Block                 object
 5   IUCR                  object
 6   Primary Type          object
 7   Description           object
 8   Location Description  object
 9   Arrest                object
 10  Domestic              object
 11  Beat                  object
 12  District              object
 13  Ward                  object
 14  Community Area        object
 15  FBI Code              object
 16  X Coordinate          object
 17  Y Coordinate          object
 18  Year                  object
 19  Updated On            object
 20  Latitude              object
 21  Longitude             object
 22  Location              object
dtypes: object(23)
memory usage: 1.4+

Our `Location description` column is missing 1,990 values. With the exception of columns `Longitude` and `Latitude`, colums 10-22 won't be used. 

### Initial Cleaning: Reducing our Dataset To Core 8 Columns

We will start with reducing our dataset to only our core 8 columns:`Case Number`, `Date`, `Block`,`Primary Type`, `Description`,`Location Description`,`X Coordinate`, `Y Coordinate`.

In [28]:
core_columns = [
    'Date',
    'Block',
    'Primary Type',
    'Description',
    'Location Description',
    'X Coordinate',
    'Y Coordinate',
]

# update our dataframe with our core eight columns
chicago_updated = chicago[core_columns].copy()

In [29]:
chicago_updated.head()

,Date,Block,Primary Type,Description,Location Description,X Coordinate,Y Coordinate
0,10/07/2008 12:39:00 PM,000XX E 75TH ST,HOMICIDE,FIRST DEGREE MURDER,ALLEY,1178207.0,1855308.0
1,10/09/2008 03:30:00 AM,048XX W POLK ST,HOMICIDE,FIRST DEGREE MURDER,STREET,1144200.0,1895857.0
2,10/09/2008 08:35:00 AM,030XX W MANN DR,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,1157314.0,1859778.0
3,10/10/2008 02:33:00 AM,052XX W CHICAGO AVE,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,1141065.0,1904824.0
4,10/10/2008 12:50:00 PM,026XX S HOMAN AVE,HOMICIDE,FIRST DEGREE MURDER,GARAGE,1154123.0,1886297.0


We can now change some of our columns names for clarity and readability.
- `Primary Type` to `Offense`,
- `Location Description` to `Incident Location` 

In [30]:
new_cols = {'Primary Type': 'Offense',
            'Description': 'Offense Description',
            'Location Description': 'Offense Location',
           }


In [31]:
chicago_updated.rename(columns=new_cols,inplace=True)

In [32]:
chicago_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7941282 entries, 0 to 1456713
Data columns (total 7 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Date                 object
 1   Block                object
 2   Offense              object
 3   Offense Description  object
 4   Offense Location     object
 5   X Coordinate         object
 6   Y Coordinate         object
dtypes: object(7)
memory usage: 484.7+ MB


In [33]:
chicago_updated.head()

,Date,Block,Offense,Offense Description,Offense Location,X Coordinate,Y Coordinate
0,10/07/2008 12:39:00 PM,000XX E 75TH ST,HOMICIDE,FIRST DEGREE MURDER,ALLEY,1178207.0,1855308.0
1,10/09/2008 03:30:00 AM,048XX W POLK ST,HOMICIDE,FIRST DEGREE MURDER,STREET,1144200.0,1895857.0
2,10/09/2008 08:35:00 AM,030XX W MANN DR,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,1157314.0,1859778.0
3,10/10/2008 02:33:00 AM,052XX W CHICAGO AVE,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,1141065.0,1904824.0
4,10/10/2008 12:50:00 PM,026XX S HOMAN AVE,HOMICIDE,FIRST DEGREE MURDER,GARAGE,1154123.0,1886297.0


We will need to change our `date` column to datetime object for easier manipulation, later on. 

In [34]:
# convert to datetime object
chicago_updated['Date'] = pd.to_datetime(chicago_updated['Date'])                                 

We can now reorder our columns to improve our flow of information.

In [ ]:
col_reorder = [
    'Date',
    'Time of Day',
    'Block',
    'Offense',
    'Offense Description',
    'Incident Location',
    'District',
    'Map Location',
    'Case ID'
]


In [ ]:
chicago_updated = chicago_updated.reindex(columns=col_reorder)

In [ ]:
chicago_updated = chicago_updated.drop(columns='Time of Day')

In [ ]:
chicago_updated.head()

It would be helpful to extract only the numerical portion of our `Block` column, and assign it to a new column, `Block Number`. We can then take the remaining address and assign it to another new column, `Street`.

In [ ]:
# search pattern representing the numerical portion of our `Block` column
pattern = r"(\d+X+)"


In [ ]:
# extract all block numbers from address
block_numbers = chicago_updated['Block'].str.extract(pattern)

In [ ]:
chicago_updated['Block Number'] = block_numbers

Now, we can create a simple function to clean and extract the street portion of the `Address` column

In [ ]:
# extracts and returns the street name
def clean_address(address):
    address = str(address)
    street_name = address.split()[1:]
    street_name = ','.join(street_name)
    street_name = street_name.replace(',',' ')
                                    
    return street_name


In [ ]:
chicago_updated.head()

In [ ]:
# extract the street name for every row in our data set
chicago_updated['Street'] = chicago_updated['Block'].apply(clean_address)

In [ ]:
chicago_updated.head()

Let's drop the old address column, and re-order our `Street` column for better flow.

In [ ]:
chicago_updated = chicago_updated.drop(columns='Block')

In [ ]:
col_reorder_2 = [
    'Date',
    'Time of Day',
    'Street',
    'Offense',
    'Offense Description',
    'Incident Location',
    'District',
    'Map Location',
    'Case ID',
    'Block Number'
]


In [ ]:
chicago_updated = chicago_updated.reindex(columns=col_reorder_2)

In [ ]:
chicago_updated.head()

Let's also drop our `Time of Day` column since it's included in our `Date` column. `Case ID` can also be dropped.

In [ ]:
chicago_updated = chicago_updated.drop(columns=['Date','Case ID'])

In [ ]:
chicago_updated.head()

### Dealing with Remaining Missing Values

Let's look at the proportion of missing values in our dataframe. 

In [ ]:
chicago_updated.isnull().sum()

We can see that there are very few missing values relative to the size of our dataset-nearly 8 million rows.`Offense`,` Street` and `Date` have no missing values which is good news.

The remaining columns with missing vlaues `Incident Location`, `District` and `Block Number` can be left as-is for now. `Map Location` contains the most missing values, and we will deal with that later on when we analyze our maps. 

In [ ]:
chicago_updated['Incident Location'].value_counts().head(5)

In [ ]:
chicago_updated.head(3)

In [ ]:
# let's add our arrest column back in!
chicago.drop_duplicates(inplace=True)

### Dealing with Duplicate Values

Let's now check for duplicate rows.


In [ ]:
chicago_updated.duplicated().sum()

We can drop our duplicate rows. 

In [ ]:
chicago_updated.drop_duplicates(inplace=True)

In [ ]:
chicago_updated.duplicated().sum()

Let's look at our missing values for our entire Dataframe. 

In [ ]:
chicago_updated.isnull().sum() / chicago_updated.shape[0] * 100

Our `Map Location` column has the most null values with 1.36%. Let's leave it for now. 

### Focusing on Violent Crimes Moving Forward

The focus of our analysis is indicators of *violent* crimes. For the purpose of our project, we will narrow our analysis to: `BATTERY`,`ASSAULT`,`ROBBERY`,`CRIM SEXUAL ASSAULT`,`HOMICIDE`. Due to the nature of our data, we will be unable to distinguish between felony and non-felony offenses in the assault and battery category.

In [ ]:
chicago_updated.head()

In [ ]:
# violent offenses 
violent_crimes = ['BATTERY','CRIM SEXUAL ASSAULT','ASSAULT','ROBBERY','HOMICIDE']

# new empty dataframe for violent offenses
chicago_violent = pd.DataFrame()

# create dataframe with only violent offenses
for crime in violent_crimes:
    chicago_violent = pd.concat([chicago_violent, chicago_updated[chicago_updated['Offense'] == crime]])

In [ ]:
chicago_violent.head()

In [ ]:
import re

In [ ]:
# small dataset to test operations
chicago_small = chicago_violent.iloc[0:2000:].copy()

### What Times are Violent Crimes Most Likely to Occur? 

When considering when violent crimes are most likely to take place, most of us would bet on the late evening hours. However, the data does not support this. According to our data, overall, violent crimes were **most likely to take place at lunchtime around 12pm, followed by 10pm-11pm. 

##### When Violent Crimes Are Most Likely to Occur

In [ ]:
chicago_violent['Date'].dt.strftime("%Ip").sort_values().value_counts().plot(kind='bar')
plt.title('Violent Crimes by Time')
plt.ylabel('Number of Occurences')
plt.xlabel('Time of Occurrence')
plt.show()

In [ ]:
# change Date to datetime object"
chicago_violent['Date'] = pd.to_datetime(chicago_violent['Date'])

### Violent Crimes in Broad Daylight are Most Common

We can now look at individual crimes and when they were most likely to occur. 

In [ ]:
chicago_violent.head()

In [ ]:
for crime in chicago_violent['Offense'].unique():
    offense_by_hour_count = chicago_violent[chicago_violent['Offense'] == crime]['Date'].dt.strftime("%Ip").value_counts()
    plt.title(crime)
    plt.plot(offense_by_hour_count)
    plt.show()

Surprising? We can see from above that much of the violent crime is committed in broad daylight! Looking at `HOMICIDES` we can see that the peak time is 12pm. With the exception of `ASSAULT`, the remaining offenses peak at similar times. 

### Violent Crime and Weather 

Is there a correlation with violent crime and temperature? To find out, we will import our weather dataset and merge in with our `chicago_violent` dataset.

In [ ]:
chicago_updated.head()

### Importing Our Weather Data

For our weather we'll import our dataset from O'Hare Airport. You can find this data set here: [noaa.gov](https://www.ncdc.noaa.gov/cdo-web/search). We are interested in `TAVG`: Average temperature for that day. We will need to convert our `DATE` column in our weather dataset to a datetime object in order to work with it later. 

In [ ]:
ohare_file = pd.read_csv('ohare_weather.csv')
ohare_weather = pd.DataFrame(ohare_file)

In [ ]:
ohare_weather.head()